監督式學習實現

In [1]:
import gym


LEFT = (1, 0)
RIGHT = (0, 1)


env = gym.make('CartPole-v0')

In [2]:
episodes = 10000
excepted_score = 100

train_x = []
train_y = []

for _ in range(episodes):
    observation = env.reset()
    
    x = []
    y = []
    
    score = 0
    
    while True:
        action = env.action_space.sample()

        x.append(observation.tolist())
        y.append(LEFT if action == 0 else RIGHT)
        
        observation, reward, done, _ = env.step(action)

        score += reward
        
        if done:
            if score >= excepted_score:
                train_x.extend(x)
                train_y.extend(y)
            
            break

print(f"從不斷隨機嘗試中, 產生了{len(train_x)}筆至少達到{excepted_score}分的訓練集")

從不斷隨機嘗試中, 產生了788筆至少達到100分的訓練集


In [3]:
import tensorflow as tf


model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='selu'),
    tf.keras.layers.Dense(128, activation='selu'),
    tf.keras.layers.Dense(2),
])
model.compile(optimizer='adam', 
              loss='mse', # mse效果優於ce
              metrics=['acc'])
model.fit(train_x, train_y, epochs=10)

Epoch 1/10
25/25 [==============================] - 0s 998us/step - loss: 0.3656 - acc: 0.5571
Epoch 2/10
25/25 [==============================] - 0s 997us/step - loss: 0.2580 - acc: 0.6028
Epoch 3/10
25/25 [==============================] - 0s 997us/step - loss: 0.2447 - acc: 0.6003
Epoch 4/10
25/25 [==============================] - 0s 960us/step - loss: 0.2474 - acc: 0.6028
Epoch 5/10
25/25 [==============================] - 0s 957us/step - loss: 0.2395 - acc: 0.5964
Epoch 6/10
25/25 [==============================] - 0s 918us/step - loss: 0.2400 - acc: 0.6041
Epoch 7/10
25/25 [==============================] - 0s 838us/step - loss: 0.2297 - acc: 0.6129
Epoch 8/10
25/25 [==============================] - 0s 917us/step - loss: 0.2390 - acc: 0.5964
Epoch 9/10
25/25 [==============================] - 0s 958us/step - loss: 0.2308 - acc: 0.6180
Epoch 10/10
25/25 [==============================] - 0s 957us/step - loss: 0.2340 - acc: 0.6091


In [4]:
import time


observation = env.reset() # observation.ndim = (4, )

score = 0

while True:
    observation = observation.reshape(1, 4)
    action = model.predict(observation) # action.ndim = (1, 2)
    action = action[0].argmax()
    observation, reward, done, _ = env.step(action)
    
    score += reward
    
    env.render()
    time.sleep(0.001)
    
    if done:
        print(f"得分: {int(score)}/200")
        break

env.close()

得分: 159/200
